# Neural Networks Image Classifier - using Keras

#### Implementing Nerual Network Classifier using Keras and evaluating the predicted classification using Sickit-Learn. 

#### Data Set

Download the data set and place it in the appropriate directory. The zip file of data set is available on university site - https://content.bellevue.edu/cst/dsc/550/id/data.zip.  The JSONL data files for this project are in reddit directory. Place the "controversial-comments.jsonl" and "categorized-comments.jsonl" in the directory.

#### Importing required modules

In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


C:\Installed\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
# Set that the color channel value will be first
K.set_image_data_format("channels_first")
# Set seed
np.random.seed(0)


In [3]:
# Set image information
channels = 1
height = 28
width = 28


In [4]:
# Load data and target from MNIST data
(data_train, target_train), (data_test, target_test) = mnist.load_data()
# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0], channels, height, width)
# Reshape test image data into features
data_test = data_test.reshape(data_test.shape[0], channels, height, width)

In [5]:
# Rescale pixel intensity to between 0 and 1
features_train = data_train / 255
features_test = data_test / 255

In [6]:
# One-hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [7]:
# Start neural network
network = Sequential()

In [8]:
# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters=64,
                        kernel_size=(5, 5),
                        input_shape=(channels, width, height),
                        activation='relu'))

# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size=(2, 2)))

# Add dropout layer
network.add(Dropout(0.5))

# Add layer to flatten input
network.add(Flatten())

# # Add fully connected layer of 128 units with a ReLU activation function
network.add(Dense(128, activation="relu"))

# Add dropout layer
network.add(Dropout(0.5))

# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes, activation="softmax"))

In [9]:
# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                    optimizer="rmsprop", # Root Mean Square Propagation
                    metrics=["accuracy"]) # Accuracy performance metric

In [10]:
# Train neural network
network.fit(features_train, # Features
                target_train, # Target
                epochs=2, # Number of epochs
                verbose=0, # Dont print description after each epoch
                batch_size=1000, # Number of observations per batch
                validation_data=(features_test, target_test)) # Data for evaluation


In [13]:
# print model summary
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 24, 24)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 12, 12)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 12, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total para

In [11]:
# Saving model

from keras.models import load_model

network.save('cnnmodel.h5')

mymodel = load_model('cnnmodel.h5')

In [12]:
#Evaluate model

score = mymodel.evaluate(features_test, target_test)
print(score)


10000/10000 [==============================] - 23s 2ms/step
[0.09943177758976818, 0.97]


# Testing for single image

In [15]:
# Grab an image from the test dataset
img = data_test[0]
print(img.shape)

(1, 28, 28)


In [17]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

(1, 1, 28, 28)


In [18]:
img_pred = network.predict(img)
print(img_pred)

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [19]:
np.argmax(img_pred[0])

7

In [23]:
actual_label = target_test[0]
actual_label


array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [25]:
np.argmax(target_test[0])

7

Predicted label matches with the test label

# Finding Accuracy of test data

In [14]:
image_pred = network.predict(data_test)
image_pred

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
np.argmax(image_pred)

7

In [28]:
predicted_labels = image_pred.argmax(axis = 1)
predicted_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [29]:
actual_labels = target_test.argmax(axis = 1)
actual_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [30]:
# for measuring accuracy, precision, recall, f1 and auc scores
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [34]:
def fn_imgclass_metrics(actual_label, predicted_label):
    """
    function that takes acutal labels and predicted labels and returns
    accuracy, auc, precision, recall and f1 scores
    average = 'weighted' for multi class classification
    """
    accuracy = accuracy_score(actual_label, predicted_label)
    precision = precision_score(actual_label, predicted_label, average = 'weighted')
    recall = recall_score(actual_label, predicted_label, average = 'weighted')
    f1 = f1_score(actual_label, predicted_label, average = 'weighted')

    return (accuracy, precision, recall, f1)

In [35]:
# calling use-defined function to calculate metrics
accuracy, precision, recall, f1 = fn_imgclass_metrics(predicted_labels, actual_labels)


In [36]:
# Printing metrics
print("Accuracy : ", accuracy)
print("Precision ", precision)
print("Recall : ", recall)
print("F1 Score ", f1)

Accuracy :  0.9693
Precision  0.9696356396808463
Recall :  0.9693
F1 Score  0.9692427415790514
